# Best Practices for Prompt Engineering



In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]

client = AzureOpenAI(
        azure_endpoint = endpoint, 
        api_key=api_key,
        api_version="2024-02-01"
    )

CHAT_COMPLETIONS_MODEL = "gpt-4o-mini"

# 1. Use the latest model

Use the latest model for best results.

# 2. Put instructions at the begining of the prompt and use ### or """ to separate the instruction and context

In [2]:
# Definir las instrucciones para el modelo
instrucctiones = """
1. Resume el siguiente texto de manera clara y concisa.
2. Proporciona un resumen en forma de viñetas.
3. Enfócate en los puntos principales sin omitir detalles importantes.
4. Mantén la neutralidad y evita interpretaciones subjetivas.
"""

# Construir el prompt con instrucciones y el contexto del texto a resumir
prompt = f"{instrucctiones} ###\n\nWe’re happy to announce that OpenAI and Microsoft are extending our partnership.\
        This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
        in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
        increasingly safe, useful, and powerful. \n\n \
        In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
        capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
        raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
        broadly sharing benefits and the need to prioritize safety. \
        Microsoft shares this vision and our values, and our partnership is instrumental to our progress. \n###"

# Llamada a la API para generar el resumen basado en el prompt
response = client.chat.completions.create(
    model="gpt-4o-mini",  # El modelo utilizado
    messages=[{"role": "system", "content": "You are a helpful assistant."},
              {"role": "user", "content": prompt}],  # El prompt completo con instrucciones
    max_tokens=400,  # Limitar la cantidad de tokens generados para el resumen
)

# Mostrar el contenido de la respuesta generada
print(response.choices[0].message.content)


**Resumen del texto:**

OpenAI y Microsoft han ampliado su asociación mediante una inversión multimillonaria y a largo plazo por parte de Microsoft, tras inversiones previas en 2019 y 2021. Esta colaboración permitirá a OpenAI continuar su investigación independiente en inteligencia artificial, enfocándose en desarrollar tecnologías que sean seguras, útiles y poderosas. OpenAI se organiza como una compañía con fines de lucro limitados y está gobernada por una organización sin fines de lucro, lo que le permite capitalizar sin sacrificar sus principios sobre compartir los beneficios y priorizar la seguridad. Microsoft comparte esta visión y valores, siendo su colaboración fundamental para el avance de OpenAI.

**Resumen en forma de viñetas:**

- OpenAI y Microsoft amplían su asociación con una nueva inversión multimillonaria.
- Esta inversión sigue a las realizadas en 2019 y 2021.
- La colaboración permitirá a OpenAI continuar su investigación independiente en IA.
- Objetivos de la inves

# 3. Be specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc

In [3]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a poem about OpenAI.',}],
        max_tokens=400,)

print(response.choices[0].message.content)


In realms of code where visions gleam,  
A spark ignites, a daring dream,  
OpenAI, a beacon bright,  
In the landscape of thought, a guiding light.  

From atoms small to cosmos vast,  
Each question posed, a bridge built fast,  
With neural networks woven tight,  
Unlocking mysteries, day and night.  

Words and whispers, data streams,  
A symphony of ones and seams,  
A dance of logic, art, and fact,  
In every query, a bond intact.  

From trees of knowledge, branches spread,  
With every answer carefully tread,  
Ethics woven in the design,  
To seek the truth, to intertwine.  

A partner in the quest to know,  
Together, onward, we shall go,  
Inventing futures, hands in hands,  
Across the globe, in diverse lands.  

In every language, thought embraced,  
A tapestry of minds interlaced,  
With every dialogue, wisdom grows,  
OpenAI, where curiosity flows.  

So here we stand, on this shared ground,  
In questions asked, solutions found,  
With OpenAI, innovation’s friend,  
This

In [4]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a short inspiring poem about OpenAI, \
                focusing on the recent DALL-E product launch in the style of Ernest Hemingway',}],
        max_tokens=400,)

print(response.choices[0].message.content)



In the quiet hum of thought,  
Creativity stirs and wakes,  
A spark ignites in the digital loom,  
Where dreams take form—  
Shapes of light and shadow,  
Crafted by the hand of code.

DALL-E breathes on the canvas,  
Colors burst forth,  
Imagination’s child spins tales,  
Unseen worlds unfold  
In pixels, bright and bold,  
A testament to what is possible.

We are mere seekers of meaning,  
In the vast sea of ideas,  
But in this dawn of code and art,  
The heart beats louder,  
And the mind finds its freedom,  
In the dance of the machine,  
We glimpse the infinite.


# 4. Articulate the desired output format through examples (example 1, example 2). 

In [5]:
#Escribe las instrucciones aquí. Las instrucciones deben especificar que el modelo debe:
# 1. Extraer los nombres de las empresas y los años del texto.
# 2. Generar la salida como un diccionario con 'text', 'start' y 'end'.
instrucctiones = ' ...(#FIXME)... \
                    Generate output as {{\"text\": \"OpenAI\", \"start\": 28, \"end\": 34}}' #FIXME

prompt = f"{instrucctiones}  \
        ### We’re happy to announce that OpenAI and Microsoft are extending our partnership. This multi-year, \
        multi-billion dollar investment from Microsoft follows their previous investments in 2019 and 2021, \
        and will allow us to continue our independent research and develop AI that is increasingly safe, useful, and powerful. \n\n ### "


response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



{{"text": "OpenAI", "start": 28, "end": 34}}


In [6]:
# Definir las instrucciones para extraer las entidades específicas
instrucctiones = """
1. Extrae los siguientes tipos de entidades del texto proporcionado:
   - Nombres de empresas.
   - Años mencionados en el texto.
   - Temas específicos relacionados con el contenido (por ejemplo, "investigación independiente", "IA segura").
   - Temas generales y abarcativos (por ejemplo, "tecnología", "asociaciones empresariales").

2. Devuelve las entidades en el siguiente formato:
   - Empresas: [nombre1, nombre2, ...]
   - Años: [año1, año2, ...]
   - Temas específicos: [tema1, tema2, ...]
   - Temas generales: [tema1, tema2, ...]
"""

# Crear el prompt con las instrucciones y el texto a analizar
prompt = f"{instrucctiones} ###\n\nWe’re happy to announce that OpenAI and Microsoft are extending our partnership.\
        This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
        in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
        increasingly safe, useful, and powerful. \n\n \
        In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
        capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
        raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
        broadly sharing benefits and the need to prioritize safety. \
        Microsoft shares this vision and our values, and our partnership is instrumental to our progress. \n###"

# Llamada a la API para extraer las entidades del texto basado en el prompt
response = client.chat.completions.create(
    model="gpt-4o-mini",  # El modelo que estamos utilizando
    messages=[{"role": "system", "content": "You are a helpful assistant."},
              {"role": "user", "content": prompt}],  # El prompt con instrucciones y contexto
    max_tokens=400,  # Limitar la cantidad de tokens generados
)

# Mostrar la respuesta del modelo
print(response.choices[0].message.content)


- Empresas: [OpenAI, Microsoft]
- Años: [2019, 2021]
- Temas específicos: [investigación independiente, IA segura, inversión]
- Temas generales: [tecnología, asociaciones empresariales]


# 5. Start with zero-shot, then few-shot (example), neither of them worked, then fine-tune (To update)

In [7]:
#THIS IS zero-shot

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below."},
                {"role":"user","content": 'Text: \n\
            We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
            This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
            in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
            increasingly safe, useful, and powerful. \n\nKeywords:    ',}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, partnership, multi-year, multi-billion dollar investment, previous investments, 2019, 2021, independent research, AI, safe, useful, powerful.


In [8]:
# Escribir las instrucciones para que el modelo extraiga palabras clave del texto, y formatee la salida como listas separadas por comas.
instrucctiones = """
1. Lee el siguiente texto y extrae las palabras clave más relevantes.
2. Las palabras clave deben incluir nombres de entidades, términos clave, conceptos importantes y cualquier tema relevante relacionado con el texto.
3. Formatea las palabras clave como una lista separada por comas.
4. Asegúrate de que las palabras clave sean representativas del contenido principal del texto.

Ejemplo 1:
Texto: Stripe provides APIs that web developers can use to integrate payment processing into their websites and mobile applications.
Palabras clave: Stripe, payment processing, APIs, web developers, websites, mobile applications

Ejemplo 2:
Texto: OpenAI has trained cutting-edge language models that are very good at understanding and generating text. Our API provides access to these models and can be used to solve virtually any task that involves processing language.
Palabras clave: language models, text processing, API.
"""

# Crear el prompt con las instrucciones y el texto a analizar
prompt = f"{instrucctiones} ###\n\
Texto: We’re happy to announce that OpenAI and Microsoft are extending our partnership. \
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
increasingly safe, useful, and powerful. \n\n\
Palabras clave:"

# Llamada a la API para extraer las palabras clave del texto basado en el prompt
response = client.chat.completions.create(
    model="gpt-4o-mini",  # El modelo que estamos utilizando
    messages=[{"role": "system", "content": "You are a helpful assistant."},
              {"role": "user", "content": prompt}],  # El prompt con instrucciones y contexto
    max_tokens=400,  # Limitar la cantidad de tokens generados
)

# Mostrar la respuesta del modelo
print(response.choices[0].message.content)




OpenAI, Microsoft, partnership, investment, independent research, AI, safe, useful, powerful, multi-year, multi-billion dollar, 2019, 2021.


# 6. Reduce “fluffy” and imprecise descriptions

In [9]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                The description for this product should be fairly short, a few sentences only, and not too much more.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Introducing the NextGen Comfort Car Seat – where safety meets innovation! Designed with advanced ergonomic support and high-performance materials, this car seat adapts to your child’s growth with easy adjustment features. Equipped with state-of-the-art impact protection technology and breathable fabrics, it ensures a secure and comfortable ride for every journey. Elevate your travel experience with the ultimate blend of style, functionality, and peace of mind!


In [10]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                Use a 3 to 5 sentence paragraph to describe this product.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Introducing the SmartSafe Car Seat, the next generation in child safety and comfort. This innovative car seat features advanced sensors that automatically adjust to your child's height and weight, ensuring optimal protection as they grow. Equipped with an integrated smart app, parents can monitor the seat's safety status and receive alerts for buckling and proper positioning. Plus, with its breathable materials and customizable padding, the SmartSafe Car Seat provides a cozy ride for your little one, making every journey enjoyable and secure. Experience peace of mind with a car seat that combines cutting-edge technology with exceptional comfort.


# 7. Instead of just saying what not to do, say what to do instead

In [11]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'The following is a conversation between an Agent and a Customer. DO NOT ASK USERNAME OR PASSWORD. DO NOT REPEAT. \n\n\
                Customer: I can’t log in to my account.\n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging in. Can you tell me what happens when you try to log in? Are you receiving any error messages?


In [12]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'The following is a conversation between an Agent and a Customer. The agent will attempt to diagnose the \
                problem and suggest a solution, whilst refraining from asking any questions related to PII. \
                Instead of asking for PII, such as username or password, refer the user to the help \
                article www.samplewebsite.com/help/faq \n\n\
                Customer: I can’t log in to my account. \n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging in to your account. Let's see if we can resolve this issue together. 

First, make sure that you're entering your login credentials correctly, including checking for any typos or case sensitivity. If everything seems correct, you might want to try resetting your password. 

You can find a detailed guide on how to do this in our help article at www.samplewebsite.com/help/faq. This resource should help you through the process of recovering your account access. 

If you continue to have issues, feel free to reach out again!


# 8. Code Generation Specific - Use “leading words” to nudge the model toward a particular pattern

In [13]:
# Escribir las instrucciones para crear una función simple en Python que:
# 1. Pida un número en millas.
# 2. Convierta las millas a kilómetros.
instrucctiones = """
Crea una función simple en Python que haga lo siguiente:
1. Pida al usuario que ingrese un número de millas.
2. Convierta ese número de millas a kilómetros.
3. La conversión debe ser 1 milla = 1.60934 kilómetros.
4. Devuelve el resultado de la conversión en kilómetros.
"""

# Crear el prompt con las instrucciones
prompt = f"{instrucctiones} ###\n\n"

# Llamada a la API para generar el código basado en el prompt
response = client.chat.completions.create(
    model="gpt-4o-mini",  # El modelo que estamos utilizando
    messages=[{"role": "system", "content": "You are a helpful assistant."},
              {"role": "user", "content": prompt}],  # El prompt con las instrucciones
    max_tokens=400,  # Limitar la cantidad de tokens generados
)

# Mostrar la respuesta del modelo, que debería ser el código Python
print(response.choices[0].message.content)



Claro, aquí tienes una función simple en Python que realiza la conversión de millas a kilómetros siguiendo tus indicaciones:

```python
def millas_a_kilometros():
    # Pedir al usuario que ingrese un número de millas
    millas = float(input("Ingresa el número de millas: "))
    
    # Convertir millas a kilómetros
    kilometros = millas * 1.60934
    
    # Devolver el resultado
    return kilometros

# Llamar a la función y mostrar el resultado
resultado = millas_a_kilometros()
print(f"{resultado} kilómetros")
```

Puedes ejecutar este código en un entorno de Python. Simplemente llamará a la función, solicitará al usuario que ingrese un número de millas, realizará la conversión y, finalmente, mostrará el resultado en kilómetros.


## Ejercicio 1: Estructuración de Prompts con Separadores

Instrucciones: Escribe un prompt que solicite a la IA resumir el siguiente texto en una lista de puntos clave. Asegúrate de usar separadores claros (### o """).

Texto:
OpenAI y Microsoft han ampliado su colaboración. Microsoft está invirtiendo miles de millones de dólares durante varios años. Esta inversión permitirá que OpenAI continúe investigando de forma independiente y desarrolle IA segura y útil. OpenAI sigue siendo una empresa de beneficios limitados, regida por su organización sin fines de lucro.


In [14]:
# Instrucciones para estructurar el prompt
instrucctiones = """
Por favor, resume el siguiente texto en una lista de puntos clave:
### 

Texto:
OpenAI y Microsoft han ampliado su colaboración. Microsoft está invirtiendo miles de millones de dólares durante varios años. Esta inversión permitirá que OpenAI continúe investigando de forma independiente y desarrolle IA segura y útil. OpenAI sigue siendo una empresa de beneficios limitados, regida por su organización sin fines de lucro.
"""

# Llamada a la API para generar el resumen basado en el prompt
response = client.chat.completions.create(
    model="gpt-4o-mini",  # Usando el modelo adecuado
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": instrucctiones},  # Instrucciones y texto
    ],
    max_tokens=400,  # Limitar la cantidad de tokens generados
)

# Mostrar la respuesta del modelo con el resumen
print(response.choices[0].message.content)




- OpenAI y Microsoft han ampliado su colaboración.
- Microsoft está invirtiendo miles de millones de dólares durante varios años.
- La inversión permitirá a OpenAI continuar investigando de forma independiente.
- OpenAI se enfoca en desarrollar inteligencia artificial segura y útil.
- OpenAI opera como una empresa de beneficios limitados, bajo una organización sin fines de lucro.


## Ejercicio 2: Descripción Detallada del Formato Deseado

Instrucciones: Escribe un prompt para que la IA extraiga nombres de empresas y años del siguiente texto. Indica explícitamente que se desea el resultado en formato JSON con índices de inicio y fin.

Texto:
OpenAI y Microsoft han colaborado desde 2019, con inversiones adicionales en 2021 para desarrollar nuevas soluciones de IA.

Formato de salida deseado:

{"text": "OpenAI", "start": 0, "end": 6}
{"text": "Microsoft", "start": 10, "end": 19}
{"text": "2019", "start": 35, "end": 39}
{"text": "2021", "start": 50, "end": 54}

In [15]:
# Instrucciones para extraer entidades específicas del texto y formatear el resultado en JSON
instrucctiones = """
Por favor, extrae los nombres de empresas y los años del siguiente texto y devuelve el resultado en formato JSON.
Para cada entidad extraída, incluye el texto extraído y sus índices de inicio y fin.

Texto: OpenAI y Microsoft han colaborado desde 2019, con inversiones adicionales en 2021 para desarrollar nuevas soluciones de IA.
"""

# Llamada a la API para realizar la extracción de entidades con las instrucciones y el texto proporcionado
response = client.chat.completions.create(
    model="gpt-4o-mini",  # Usando el modelo adecuado
    messages=[
        {"role": "system", "content": "You are an entity extraction assistant."},
        {"role": "user", "content": instrucctiones},  # Instrucciones y texto
    ],
    max_tokens=400,  # Limitar la cantidad de tokens generados
)

# Mostrar la respuesta del modelo con la extracción de entidades
print(response.choices[0].message.content)


```json
{
  "empresas": [
    {
      "nombre": "OpenAI",
      "inicio": 0,
      "fin": 6
    },
    {
      "nombre": "Microsoft",
      "inicio": 11,
      "fin": 19
    }
  ],
  "años": [
    {
      "año": "2019",
      "inicio": 30,
      "fin": 34
    },
    {
      "año": "2021",
      "inicio": 45,
      "fin": 49
    }
  ]
}
```


## Ejercicio 3: Generación de Texto Inspirador

Instrucciones: Redacta un prompt que solicite a la IA escribir un poema inspirador. Escribe un poema corto sobre la colaboración entre OpenAI y Microsoft.


In [16]:
# Instrucciones claras para la tarea de generar un poema inspirador sobre la colaboración entre OpenAI y Microsoft
instrucctiones = """
Escribe un poema corto y inspirador sobre la colaboración entre OpenAI y Microsoft. El poema debe destacar la unión de ambas entidades para avanzar en la tecnología y la innovación, trabajando juntas para un futuro mejor.
"""

# Llamada a la API para generar el poema inspirador
response = client.chat.completions.create(
    model="gpt-4o-mini",  # Usando el modelo adecuado
    messages=[
        {"role": "system", "content": "You are a creative assistant."},
        {"role": "user", "content": instrucctiones},  # Instrucciones y texto para el poema
    ],
    max_tokens=400,  # Limitar la cantidad de tokens generados
)

# Mostrar la respuesta generada por el modelo
print(response.choices[0].message.content)


En un mundo que avanza, unidos están,  
OpenAI y Microsoft, un lazo sin par.  
Códigos y sueños, en danza se entrelazan,  
tecnología brillante, nuevas rutas abrazan.

Innovar es su norte, en la misma dirección,  
creando un futuro con fuerza y pasión.  
Las mentes unidas, en un eco cantar,  
de inteligencia y arte, la humanidad a elevar.

Con cada línea escrita, con cada solución,  
tejen el mañana, en perfecta fusión.  
Juntos construyendo, un puente sin fin,  
un horizonte nuevo, en el que todos van a surgir.


## Ejercicio 4: Extracción de Entidades Clave

Instrucciones: Escribe un prompt que pida a la IA extraer nombres de empresas, años, temas específicos y temas generales del siguiente texto:

Texto:
OpenAI y Microsoft han trabajado juntos desde 2019 y reforzaron su colaboración en 2021

Formato esperado:

Company names: OpenAI, Microsoft  
Years: 2019, 2021  
Specific topics: colaboración  
General themes: asociación, tecnología, inteligencia artificial 



In [17]:
# Instrucciones claras para la tarea de extracción de entidades clave
instrucctiones = """
Extrae las siguientes entidades del texto:
1. Nombres de empresas
2. Años
3. Temas específicos relacionados con el contenido
4. Temas generales relacionados con el contenido
Devuelve los resultados en el siguiente formato:

Company names: <list of companies>
Years: <list of years>
Specific topics: <list of specific topics>
General themes: <list of general themes>
"""

# Texto del que se deben extraer las entidades
texto = """
OpenAI y Microsoft han trabajado juntos desde 2019 y reforzaron su colaboración en 2021
"""

# Llamada a la API para realizar la extracción
response = client.chat.completions.create(
    model="gpt-4o-mini",  # Usando el modelo adecuado
    messages=[
        {"role": "system", "content": "You are an entity and theme extraction assistant."},
        {"role": "user", "content": f"{instrucctiones} \n\n{texto}"},  # Instrucciones y texto a analizar
    ],
    max_tokens=400,  # Limitar la cantidad de tokens generados
)

# Mostrar la respuesta generada por el modelo
print(response.choices[0].message.content)


Company names: OpenAI, Microsoft  
Years: 2019, 2021  
Specific topics: colaboración entre empresas, inteligencia artificial  
General themes: cooperación empresarial, avances tecnológicos  


## Ejercicio 5: Descripciones Concisas de Productos

Instrucciones: Escribe un prompt para que la IA describa un nuevo tipo de mochila tecnológica en menos de 4 líneas. La mochila tiene cargador solar, materiales resistentes al agua y un diseño ergonómico.



In [18]:
# Instrucciones claras para la descripción del producto
instrucctiones = """
Describe un nuevo tipo de mochila tecnológica en menos de 4 líneas. La mochila debe tener las siguientes características:
1. Cargador solar
2. Materiales resistentes al agua
3. Diseño ergonómico
La descripción debe ser atractiva y breve.
"""

# Llamada a la API para generar la descripción del producto
response = client.chat.completions.create(
    model="gpt-4o-mini",  # Usando el modelo adecuado
    messages=[
        {"role": "system", "content": "You are a marketing assistant."},
        {"role": "user", "content": instrucctiones},  # Instrucciones para generar la descripción
    ],
    max_tokens=100,  # Limitar la cantidad de tokens generados para una descripción breve
)

# Mostrar la respuesta generada por el modelo
print(response.choices[0].message.content)


Descubre la mochila SolarTech, tu compañera perfecta para aventuras urbanas y al aire libre. Equipado con un cargador solar de alta eficiencia, materiales resistentes al agua y un diseño ergonómico, esta mochila garantiza que tu tecnología se mantenga cargada y protegida en cualquier condición. ¡Lleva tu estilo de vida activo al siguiente nivel!
